In [1]:
!wget https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf
!wget https://raw.githubusercontent.com/IlyaGusev/rulm/master/self_instruct/src/interact_mistral_llamacpp.py

--2024-02-18 15:22:40--  https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/79/b3/79b3fc4694b2c3a22273003a1de570f145c14f0586c212c28c28e302adf5d3d6/2798f33ff63c791a21f05c1ee9a10bc95630b17225c140c197188a3d5cf32644?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-q4_K.gguf%3B+filename%3D%22model-q4_K.gguf%22%3B&Expires=1708528960&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODUyODk2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83OS9iMy83OWIzZmM0Njk0YjJjM2EyMjI3MzAwM2ExZGU1NzBmMTQ1YzE0ZjA1ODZjMjEyYzI4YzI4ZTMwMmFkZjVkM2Q2LzI3OThmMzNmZjYzYzc5MWEyMWYwNWMxZWU5YTEwYmM5NTYzMGIxNzIyNWMxNDBjMTk3MTg4YTNkNWNmMzI2

In [2]:
%%capture
!pip install llama-cpp-python
!pip install faker
!pip install openai
!pip install tiktoken
!pip install cohere
!pip install accelerate
!pip install huggingface-hub
!pip install auto-gptq
!pip install optimum
!pip install fire

In [3]:
import string
import re
import random

from llama_cpp import Llama
from tqdm.auto import tqdm
from faker import Faker
import pandas as pd

In [4]:
model_path="/content/model-q4_K.gguf"
llm = Llama(
  model_path=model_path,
  n_ctx=2048,
  n_gpu_layers=-1
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/model-q4_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
l

In [5]:
def randomword(length):
   """Generate random string of length {length}
      in order to create the document id"""
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))

def generate_address():
    """Generates a random Russian address with Faker.
       Might be expanded for the sake of diversity."""
    language = "ru_RU"
    fake = Faker(language)
    return fake.address()

In [6]:
prompt = """
    Ты живешь по адресу {address}.
    Тебе необходимо написать короткое эссе про твой адрес.
    Пожлауйста, напиши сначала о том, где ты живешь и почему тебе нравится или не нравится там жить.
    Затем расскажи подробности про свое любимое место в мире, и где оно находится.
    НЕ ДАВАЙ НИКАКУЮ ЛИЧНУЮ ИНФОРМАЦИЮ О СЕБЕ КРОМЕ АДРЕСА.
""".replace("\n ", "")

for i in tqdm(range(1)):
    # Get document ID
    id = randomword(12)
    # Get address
    address = generate_address()
    # Fill in prompt
    prompt_ = prompt.format(address=address)
    # Replace duplicate whitespaces
    prompt_ = re.sub(" +", " ", prompt_)
    # Fill in Saiga template
    command = f"<s>[INST] {prompt_} [/INST]"
    output = llm(
        command,
        max_tokens=768,
        stop=["</s>"],
        echo=True,
        temperature=0.6
    )

    # Collect generated PII to label
    label_dict={"STREET_ADDRESS": [address]}

    # Collect data and append
    data=[id, output["choices"][0]["text"].split("[/INST]")[1], label_dict]
    pd.DataFrame(
        [data],
        columns=["document", "text", "label_dict"]
    ).to_csv("pii_address.csv", mode="a", header=False, index=False)

  0%|          | 0/1 [00:00<?, ?it/s]


llama_print_timings:        load time =   95545.10 ms
llama_print_timings:      sample time =     241.84 ms /   328 runs   (    0.74 ms per token,  1356.27 tokens per second)
llama_print_timings: prompt eval time =   95544.58 ms /   189 tokens (  505.53 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =  231722.05 ms /   327 runs   (  708.63 ms per token,     1.41 tokens per second)
llama_print_timings:       total time =  329082.28 ms /   516 tokens
